**Introduction**
In this raw dataset, 14 variables are used to deduce whether or not a patient has heart disease from 0 (absence) and 1-4 (presence). Using the processed Cleveland dataset I will apply the variables.
During this exploration, I seek to deduce and predict whether patients have heart disease, as well as what the relationship between the three variables is in contributing to the presence or absence of heart disease.

**Methods**
I have chosen to use the "trestbps" and "col" variables to predict whether or not a patient will have heart disease (column "num"). These variables were chosen because there are possibilities of having different numbers instead of ones that can only be 0 or 1, which deems them to have equal probability regardless of the classifier and thus rendering it irrelevant. In order to do so, I will create a classification model using the K-nearest neighbors model. First, I have cleaned the data so that all the unknown values were filtered out, and only the three columns I'm interested in remain. Then, I split the data into training and testing sets, which I will use to predict the presence or absence of heart disease by creating a recipe and a workflow.

**Expected results** 
From previous knowledge, I expect that having a higher blood pressure and cholesterol makes it more likely for a patient to have heart disease. After knowing the relationships between the blood pressure, cholesterol, and presence/absence of heart disease, the findings can be used to determine the likelihood for a patient to have the disease, and if they need to change any aspect of their lifestyle to reduce that likelihood. 